In [108]:
import os
import sys; sys.path.append("~/HHbbgg_ETH_devel/Training/python") # to load packages
import training_utils as utils
import numpy as np
reload(utils)
import preprocessing_utils as preprocessing
reload(preprocessing)
import plotting_utils as plotting
reload(plotting)
import optimization_utils as optimization
reload(optimization)
import postprocessing_utils as postprocessing
reload(postprocessing)


<module 'postprocessing_utils' from '/mnt/t3nfs01/data01/shome/micheli/HHbbgg_ETH_devel/Training/python/postprocessing_utils.pyc'>

In [109]:
reload(utils)
reload(preprocessing)
reload(plotting)
reload(optimization)
reload(postprocessing)
ntuples = '20170620'
# "%" sign allows to interpret the rest as a system command
%env data=$utils.IO.ldata$ntuples
files = ! ls $data | sort -t_ -k 3 -n
#pick only SM
signal = [s for s in files if "GluGluToHHTo2B2G_node_SM" in s]
diphotonJets = [s for s in files if "DiPhotonJetsBox_" in s]
gJets_lowPt = [s for s in files if "GJet_Pt-20to40" in s]
gJets_highPt = [s for s in files if "GJet_Pt-40" in s]
ggH = [s for s in files if "GluGluHToGG" in s]
vbf = [s for s in files if "VBFHToGG" in s]
VH = [s for s in files if "VHToGG" in s]
bbH = [s for s in files if "bbHToGG_M-125_4FS_yb2" in s] #is this correct?
ttH = [s for s in files if "ttHToGG" in s]

Data= [s for s in files if "DoubleEG" in s]

utils.IO.add_signal(ntuples,signal,1)
print diphotonJets
utils.IO.add_background(ntuples,diphotonJets,-1)
utils.IO.add_background(ntuples,gJets_lowPt,-2)
utils.IO.add_background(ntuples,gJets_highPt,-2)
utils.IO.add_background(ntuples,ggH,-3)
utils.IO.add_background(ntuples,vbf,-4)
utils.IO.add_background(ntuples,VH,-5)
utils.IO.add_background(ntuples,bbH,-6)
utils.IO.add_background(ntuples,ttH,-7)

nBkg = len(utils.IO.backgroundName)

utils.IO.add_data(ntuples,Data,-10)

#add all nodes
nodes = []
for i in range(2,14):
    nodes.append([s for s in files if "GluGluToHHTo2B2G_node_"+str(i) in s])
for i in range(len(utils.IO.backgroundName),len(utils.IO.backgroundName)+len(nodes)):
    utils.IO.add_background(ntuples,nodes[i-nBkg],-i)
    
    
    

for i in range(len(utils.IO.backgroundName)):        
    print "using background file n."+str(i)+": "+utils.IO.backgroundName[i]
for i in range(len(utils.IO.signalName)):    
    print "using signal file n."+str(i)+": "+utils.IO.signalName[i]
print "using data file: "+ utils.IO.dataName[0]


env: data=/mnt/t3nfs01/data01/shome/micheli/HHbbgg_ETH/root_files/20170620
['output_DiPhotonJetsBox_MGG-80toInf_13TeV-Sherpa.root']
using background file n.0: /mnt/t3nfs01/data01/shome/micheli/HHbbgg_ETH/root_files/20170620/output_DiPhotonJetsBox_MGG-80toInf_13TeV-Sherpa.root
using background file n.1: /mnt/t3nfs01/data01/shome/micheli/HHbbgg_ETH/root_files/20170620/output_GJet_Pt-20to40_DoubleEMEnriched_MGG-80toInf_TuneCUETP8M1_13TeV_Pythia8.root
using background file n.2: /mnt/t3nfs01/data01/shome/micheli/HHbbgg_ETH/root_files/20170620/output_GJet_Pt-40toInf_DoubleEMEnriched_MGG-80toInf_TuneCUETP8M1_13TeV_Pythia8.root
using background file n.3: /mnt/t3nfs01/data01/shome/micheli/HHbbgg_ETH/root_files/20170620/output_GluGluHToGG_M-125_13TeV_powheg_pythia8.root
using background file n.4: /mnt/t3nfs01/data01/shome/micheli/HHbbgg_ETH/root_files/20170620/output_VBFHToGG_M-125_13TeV_powheg_pythia8.root
using background file n.5: /mnt/t3nfs01/data01/shome/micheli/HHbbgg_ETH/root_files/201706

In [110]:
#use noexpand for root expressions, it needs this file https://github.com/ibab/root_pandas/blob/master/root_pandas/readwrite.py
#all variables
#branch_names = 'leadingJet_bDis,subleadingJet_bDis,noexpand:fabs(CosThetaStar_CS),noexpand:fabs(CosTheta_bb),noexpand:fabs(CosTheta_gg),noexpand:diphotonCandidate.Pt()/diHiggsCandidate.M(),noexpand:dijetCandidate.Pt()/diHiggsCandidate.M(),customLeadingPhotonIDMVA,customSubLeadingPhotonIDMVA,leadingPhotonSigOverE,subleadingPhotonSigOverE,sigmaMOverMDecorr,DiJetDiPho_DR'.split(",")
#noDR(hh)
#branch_names = 'leadingJet_bDis,subleadingJet_bDis,noexpand:fabs(CosThetaStar_CS),noexpand:fabs(CosTheta_bb),noexpand:fabs(CosTheta_gg),noexpand:diphotonCandidate.Pt()/diHiggsCandidate.M(),noexpand:dijetCandidate.Pt()/diHiggsCandidate.M(),customLeadingPhotonIDMVA,customSubLeadingPhotonIDMVA,leadingPhotonSigOverE,subleadingPhotonSigOverE,sigmaMOverMDecorr'.split(",")
#noDR(hh) + minDR(gJ)
branch_names = 'leadingJet_bDis,subleadingJet_bDis,noexpand:fabs(CosThetaStar_CS),noexpand:fabs(CosTheta_bb),noexpand:fabs(CosTheta_gg),noexpand:diphotonCandidate.Pt()/diHiggsCandidate.M(),noexpand:dijetCandidate.Pt()/diHiggsCandidate.M(),customLeadingPhotonIDMVA,customSubLeadingPhotonIDMVA,leadingPhotonSigOverE,subleadingPhotonSigOverE,sigmaMOverMDecorr,PhoJetMinDr'.split(",")
#### Trying to add Mjj as variable
#branch_names = 'leadingJet_bDis,subleadingJet_bDis,noexpand:fabs(CosThetaStar_CS),noexpand:fabs(CosTheta_bb),noexpand:fabs(CosTheta_gg),noexpand:diphotonCandidate.Pt()/diHiggsCandidate.M(),noexpand:dijetCandidate.Pt()/diHiggsCandidate.M(),customLeadingPhotonIDMVA,customSubLeadingPhotonIDMVA,leadingPhotonSigOverE,subleadingPhotonSigOverE,sigmaMOverMDecorr,noexpand:dijetCandidate.M()'.split(",")
#### Trying to add Mjj as variable, keeping DR
#branch_names = 'leadingJet_bDis,subleadingJet_bDis,noexpand:fabs(CosThetaStar_CS),noexpand:fabs(CosTheta_bb),noexpand:fabs(CosTheta_gg),noexpand:diphotonCandidate.Pt()/diHiggsCandidate.M(),noexpand:dijetCandidate.Pt()/diHiggsCandidate.M(),customLeadingPhotonIDMVA,customSubLeadingPhotonIDMVA,leadingPhotonSigOverE,subleadingPhotonSigOverE,sigmaMOverMDecorr,DiJetDiPho_DR,noexpand:dijetCandidate.M()'.split(",")
#### Trying to add Mjj as variable, removing DR, adding minDr(Gamma,Jet)
#branch_names = 'leadingJet_bDis,subleadingJet_bDis,noexpand:fabs(CosThetaStar_CS),noexpand:fabs(CosTheta_bb),noexpand:fabs(CosTheta_gg),noexpand:diphotonCandidate.Pt()/diHiggsCandidate.M(),noexpand:dijetCandidate.Pt()/diHiggsCandidate.M(),customLeadingPhotonIDMVA,customSubLeadingPhotonIDMVA,leadingPhotonSigOverE,subleadingPhotonSigOverE,sigmaMOverMDecorr,PhoJetMinDr,noexpand:dijetCandidate.M()'.split(",")


#std variables
#branch_names = 'leadingJet_bDis,subleadingJet_bDis,noexpand:fabs(CosThetaStar_CS),noexpand:fabs(CosTheta_bb),noexpand:fabs(CosTheta_gg),noexpand:diphotonCandidate.Pt()/diHiggsCandidate.M(),noexpand:dijetCandidate.Pt()/diHiggsCandidate.M()'.split(",")
branch_names = [c.strip() for c in branch_names]
print branch_names

import pandas as pd
import root_pandas as rpd

# no need to shuffle here, we just count events
preprocessing.set_signals_and_backgrounds("bbggSelectionTree",branch_names,shuffle=False)
X_bkg,y_bkg,weights_bkg,X_sig,y_sig,weights_sig=preprocessing.set_variables(branch_names)

X_data,y_data,weights_data = preprocessing.set_data("bbggSelectionTree",branch_names)
X_data,y_data,weights_data = preprocessing.clean_signal_events_single_dataset(X_data,y_data,weights_data)

#bbggTrees have by default signal and CR events, let's be sure that we clean it
X_bkg,y_bkg,weights_bkg,X_sig,y_sig,weights_sig=preprocessing.clean_signal_events(X_bkg,y_bkg,weights_bkg,X_sig,y_sig,weights_sig)

#np.savetxt('/tmp/micheli/xbkg.txt',X_bkg)
#preprocessing.adjust_and_compress(utils.IO.background_df[0]).to_csv('/tmp/micheli/bkg.csv')


['leadingJet_bDis', 'subleadingJet_bDis', 'noexpand:fabs(CosThetaStar_CS)', 'noexpand:fabs(CosTheta_bb)', 'noexpand:fabs(CosTheta_gg)', 'noexpand:diphotonCandidate.Pt()/diHiggsCandidate.M()', 'noexpand:dijetCandidate.Pt()/diHiggsCandidate.M()', 'customLeadingPhotonIDMVA', 'customSubLeadingPhotonIDMVA', 'leadingPhotonSigOverE', 'subleadingPhotonSigOverE', 'sigmaMOverMDecorr', 'DiJetDiPho_DR']


In [111]:
# load the model from disk
from sklearn.externals import joblib
#loaded_model = joblib.load(os.path.expanduser('~/HHbbgg_ETH_devel/Training/output_files/allMC_stdVariables.pkl'))
#loaded_model = joblib.load(os.path.expanduser('~/HHbbgg_ETH_devel/Training/output_files/allMC_resWeighting_F_noDR.pkl'))
#loaded_model = joblib.load(os.path.expanduser('~/HHbbgg_ETH_devel/Training/output_files/allMC_resWeighting_F.pkl'))
#loaded_model = joblib.load(os.path.expanduser('~/HHbbgg_ETH_devel/Training/output_files/allMC_resWeighting_F_noDR_minDRGJet.pkl'))
#loaded_model = joblib.load(os.path.expanduser('~/HHbbgg_ETH_devel/Training/output_files/allMC_resWeighting_F_noDR_dijetMass.pkl'))
#loaded_model = joblib.load(os.path.expanduser('~/HHbbgg_ETH_devel/Training/output_files/allMC_resWeighting_F_withDR_dijetMass.pkl'))
#loaded_model = joblib.load(os.path.expanduser('~/HHbbgg_ETH_devel/Training/output_files/allMC_resWeighting_F_noDR_minDRGJet_dijetMass.pkl'))
#loaded_model = joblib.load(os.path.expanduser('~/HHbbgg_ETH_devel/Training/output_files/allMC_resWeighting_F_BJetsSamples.pkl'))
#loaded_model = joblib.load(os.path.expanduser('~/HHbbgg_ETH_devel/Training/output_files/allMC_resWeighting_F_BJetsSamples_2.pkl'))
#loaded_model = joblib.load(os.path.expanduser('~/HHbbgg_ETH_devel/Training/output_files/allMC_resWeighting_F_BJetsSamples_3.pkl'))
#loaded_model = joblib.load(os.path.expanduser('~/HHbbgg_ETH_devel/Training/output_files/allMC_resWeighting_F_resH.pkl'))
#loaded_model = joblib.load(os.path.expanduser('~/HHbbgg_ETH_devel/Training/output_files/allMC_resWeighting_F_ttH.pkl'))
#loaded_model = joblib.load(os.path.expanduser('~/HHbbgg_ETH_devel/Training/output_files/allMC_resWeighting_F_ggH_ttH.pkl'))
#loaded_model = joblib.load(os.path.expanduser('~/HHbbgg_ETH_devel/Training/output_files/allMC_resWeighting_F_new.pkl'))
#loaded_model = joblib.load(os.path.expanduser('~/HHbbgg_ETH_devel/Training/output_files/allMC_resWeighting_F_ggH_reweight.pkl'))
#loaded_model = joblib.load(os.path.expanduser('~/HHbbgg_ETH_devel/Training/output_files/allMC_resWeighting_F_ggH_ttH_reweight.pkl'))
loaded_model = joblib.load(os.path.expanduser('~/HHbbgg_ETH_devel/Training/output_files/allMC_resWeighting_F_ggH_reweight_scale10.pkl'))

bkg = []
for i in range(0,len(utils.IO.backgroundName)-1): 
    bkg.append(X_bkg[y_bkg ==-i-1])


Y_pred_sig = loaded_model.predict_proba(X_sig)[:,loaded_model.n_classes_-1].astype(np.float64)
Y_pred_bkg = []
for i in range(0,len(utils.IO.backgroundName)-1):  
    print i
    Y_pred_bkg.append(loaded_model.predict_proba(bkg[i])[:,loaded_model.n_classes_-1].astype(np.float64))

Y_pred_data = loaded_model.predict_proba(X_data)[:,loaded_model.n_classes_-1].astype(np.float64)
print Y_pred_data 



0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
[ 0.00192842  0.22193955  0.00448152 ...,  0.02485011  0.04200699
  0.58325577]


In [112]:
#define MVA cut and additional variables needed
import os
MVA_Cut = 0.92
additionalCut_names = 'noexpand:diphotonCandidate.M(),noexpand:dijetCandidate.M(),MX,isSignal'.split(",")
outTag = '20170828_optimizedCuts_ggH_reweight_scale10'
outDir=os.path.expanduser("~/HHbbgg_ETH_devel/outfiles/"+outTag)
if not os.path.exists(outDir):
    os.mkdir(outDir)

In [113]:
#### here counting events only in dijet mass window######
#print utils.IO.signal_df[0]['proc']
#print utils.IO.signal_df[0].index
sig_count_df = rpd.read_root(utils.IO.signalName[0],"bbggSelectionTree", columns = branch_names+additionalCut_names)
preprocessing.define_process_weight(sig_count_df,utils.IO.sigProc[0],utils.IO.signalName[0])

#nTot is a multidim vector with all additional variables, dictVar is a dictionary associating a name of the variable
#to a position in the vector
nTot,dictVar = postprocessing.stackFeatures(sig_count_df,branch_names+additionalCut_names)
#apply isSignal cleaning
nCleaned = nTot[np.where(nTot[:,dictVar['weight']]!=0),:][0]
print "Y_pred"
print Y_pred_sig.shape

#it's a stupid way to save also mva output after cuts, more efficient solutions should be found
Y_pred_total = np.column_stack((nCleaned,Y_pred_sig))

nCleaned_MVA = nCleaned[np.where(Y_pred_sig>MVA_Cut)]
#apply massWindow +- 1.5
print nCleaned_MVA.shape
nCleaned_massWindow = postprocessing.cutInvariantMass(nCleaned_MVA,dictVar['diphotonCandidate.M()'],123.5000,126.5000)
nEvts = sum(nCleaned_massWindow[:,dictVar['weight']])
print "signal events with 36 fb-1: "+str(nEvts)
#import matplotlib.pyplot as plt
#plt.hist(nCleaned_massWindow[:,2], bins=range(int(min(nCleaned_massWindow[:,2])), int(max(nCleaned_massWindow[:,2])) + 5, 5))
#plt.show()
nCleaned_mdijet = postprocessing.cutInvariantMass(nCleaned_massWindow,dictVar['dijetCandidate.M()'],100.,140.)
nEvts_Cut = sum(nCleaned_mdijet[:,dictVar['weight']])
print "after cutting in dijet mass:"+str(nEvts_Cut)

processPath=os.path.expanduser('~/HHbbgg_ETH_devel/outfiles/')+outTag+'/'+utils.IO.signalName[0].split("/")[len(utils.IO.signalName[0].split("/"))-1].replace("output_","")
postprocessing.saveTree(processPath,dictVar,nCleaned_mdijet)

processPath=os.path.expanduser('~/HHbbgg_ETH_devel/outfiles/')+outTag+'/'+"SigWS_highMass.root"
nCleaned_MVA_2 = Y_pred_total[np.where(Y_pred_sig>MVA_Cut)]
#

nCleaned_massWindow_2 = postprocessing.cutInvariantMass(nCleaned_MVA_2,dictVar['diphotonCandidate.M()'],100,180)

nCleaned_mdijet_2 = postprocessing.cutInvariantMass(nCleaned_massWindow_2,dictVar['dijetCandidate.M()'],100.,140.)
nCleaned_MX = postprocessing.applyCut(nCleaned_mdijet_2,dictVar['MX'],350)


Y_pred_cut=nCleaned_MX[:,nCleaned_MX.shape[1]-1].astype(np.float64)


postprocessing.saveTree(processPath,dictVar,nCleaned_MX,Y_pred_cut,nameTree="reducedTree_sig")

processPath=os.path.expanduser('~/HHbbgg_ETH_devel/outfiles/')+outTag+'/'+utils.IO.signalName[0].split("/")[len(utils.IO.signalName[0].split("/"))-1].replace("output_","").replace(".root","")+"_preselection"+".root"
postprocessing.saveTree(processPath,dictVar,nCleaned,Y_pred_sig)

processPath=os.path.expanduser('~/HHbbgg_ETH_devel/outfiles/')+outTag+'/'+utils.IO.signalName[0].split("/")[len(utils.IO.signalName[0].split("/"))-1].replace("output_","").replace(".root","")+"_preselection_diffNaming"+".root"
postprocessing.saveTree(processPath,dictVar,nCleaned,Y_pred_sig,nameTree="reducedTree_sig")


Y_pred
(18646,)
(12036, 18)
signal events with 36 fb-1: 7.02346408768
after cutting in dijet mass:4.78391240235


In [114]:
## do gJets not in the loop since they have two samples for one process
bkg_1_count_df = rpd.read_root(utils.IO.backgroundName[1],"bbggSelectionTree", columns = branch_names+additionalCut_names)
preprocessing.define_process_weight(bkg_1_count_df,utils.IO.bkgProc[1],utils.IO.backgroundName[1])

crazySF_20=25
nTot,dictVar = postprocessing.stackFeatures(bkg_1_count_df,branch_names+additionalCut_names,SF=crazySF_20)

print nTot.shape

bkg_2_count_df = rpd.read_root(utils.IO.backgroundName[2],"bbggSelectionTree", columns = branch_names+additionalCut_names)
preprocessing.define_process_weight(bkg_2_count_df,utils.IO.bkgProc[2],utils.IO.backgroundName[2])

crazySF_40=3
nTot_2,dictVar = postprocessing.stackFeatures(bkg_2_count_df,branch_names+additionalCut_names,SF=crazySF_40)


nTot_3 = np.concatenate((nTot,nTot_2))

print nTot_3.shape
#apply isSignal cleaning
nCleaned = nTot_3[np.where(nTot_3[:,dictVar['weight']]!=0),:][0]
print "nCleaned"
print nCleaned.shape
#apply MVA cut at 90% eff
nCleaned_MVA = nCleaned[np.where(Y_pred_bkg[1]>MVA_Cut)]
print nCleaned_MVA.shape
#apply massWindow +- 1.5
nCleaned_massWindow = postprocessing.cutInvariantMass(nCleaned_MVA,dictVar['diphotonCandidate.M()'],123.5000,126.5000)
print nCleaned_massWindow.shape
nEvts = sum(nCleaned_massWindow[:,dictVar['weight']])
print utils.IO.backgroundName[2].split("/")[len(utils.IO.backgroundName[2].split("/"))-1].replace(".root","").replace("output_","")+" events with 36 fb-1: "+str(nEvts)
nCleaned_mdijet = postprocessing.cutInvariantMass(nCleaned_massWindow,dictVar['dijetCandidate.M()'],100.,140.)
print nCleaned_mdijet.shape
nEvts_Cut = sum(nCleaned_mdijet[:,dictVar['weight']])
print "after cutting in dijet mass:"+str(nEvts_Cut)

#np.savetxt('/tmp/micheli/prova7.txt',nCleaned_mdijet)
#preprocessing.adjust_and_compress(bkg_2_count_df).to_csv('/tmp/micheli/background_7_2.csv') 
processPath=os.path.expanduser('~/HHbbgg_ETH_devel/outfiles/')+outTag+'/'+utils.IO.backgroundName[1].split("/")[len(utils.IO.backgroundName[1].split("/"))-1].replace("output_","").replace("_Pt-20to40","")
postprocessing.saveTree(processPath,dictVar,nCleaned_mdijet)


processPath=(os.path.expanduser('~/HHbbgg_ETH_devel/outfiles/')+outTag+'/'+utils.IO.backgroundName[1].split("/")[len(utils.IO.backgroundName[1].split("/"))-1].replace("output_","").replace(".root","")+"_preselection"+".root").replace("_Pt-20to40","")
postprocessing.saveTree(processPath,dictVar,nCleaned,Y_pred_bkg[1])

processPath=(os.path.expanduser('~/HHbbgg_ETH_devel/outfiles/')+outTag+'/'+utils.IO.backgroundName[1].split("/")[len(utils.IO.backgroundName[1].split("/"))-1].replace("output_","").replace(".root","")+"_preselection_diffNaming"+".root").replace("_Pt-20to40","")
postprocessing.saveTree(processPath,dictVar,nCleaned,Y_pred_bkg[1],nameTree="reducedTree_bkg_2")




(2590, 18)
(63259, 18)
nCleaned
(22278, 18)
(16, 18)
(1, 18)
GJet_Pt-40toInf_DoubleEMEnriched_MGG-80toInf_TuneCUETP8M1_13TeV_Pythia8 events with 36 fb-1: 0.501821236393
(0, 18)
after cutting in dijet mass:0


In [115]:
for iProcess in range(0,len(utils.IO.backgroundName)):
    ##gJets which are two samples for one process are skipped
    iSample=iProcess
    if iProcess == 1 or iProcess ==2:
            continue
    if iProcess > 2:
        iSample = iProcess-1
    
    print "Processing sample: "+str(iProcess)
    bkg_count_df = rpd.read_root(utils.IO.backgroundName[iProcess],"bbggSelectionTree", columns = branch_names+additionalCut_names)
    preprocessing.define_process_weight(bkg_count_df,utils.IO.bkgProc[iProcess],utils.IO.backgroundName[iProcess])

    crazySF=1
    ##scale diphoton + jets
    if iProcess == 0:
        crazySF=1.45
    nTot,dictVar = postprocessing.stackFeatures(bkg_count_df,branch_names+additionalCut_names,SF=crazySF)


    nCleaned = nTot[np.where(nTot[:,dictVar['weight']]!=0),:][0]
    print "nCleaned"
    print nCleaned.shape
    #apply MVA cut at 90% eff
    nCleaned_MVA = nCleaned[np.where(Y_pred_bkg[iSample]>MVA_Cut)]
    print nCleaned_MVA.shape
    #apply massWindow +- 1.5
    nCleaned_massWindow = postprocessing.cutInvariantMass(nCleaned_MVA,dictVar['diphotonCandidate.M()'],123.5000,126.5000)
    print nCleaned_massWindow.shape
    nEvts = sum(nCleaned_massWindow[:,dictVar['weight']])
    print utils.IO.backgroundName[iProcess].split("/")[len(utils.IO.backgroundName[1].split("/"))-1].replace(".root","").replace("output_","")+" events with 36 fb-1: "+str(nEvts)
    nCleaned_mdijet = postprocessing.cutInvariantMass(nCleaned_massWindow,dictVar['dijetCandidate.M()'],100.,140.)
    print nCleaned_mdijet.shape
    nEvts_Cut = sum(nCleaned_mdijet[:,dictVar['weight']])
    print "after cutting in dijet mass:"+str(nEvts_Cut)


    processPath=os.path.expanduser('~/HHbbgg_ETH_devel/outfiles/')+outTag+'/'+utils.IO.backgroundName[iProcess].split("/")[len(utils.IO.backgroundName[7].split("/"))-1].replace("output_","")
    postprocessing.saveTree(processPath,dictVar,nCleaned_mdijet)


    processPath=os.path.expanduser('~/HHbbgg_ETH_devel/outfiles/')+outTag+'/'+utils.IO.backgroundName[iProcess].split("/")[len(utils.IO.backgroundName[7].split("/"))-1].replace("output_","").replace(".root","")+"_preselection"+".root"
    postprocessing.saveTree(processPath,dictVar,nCleaned,Y_pred_bkg[iSample])
    
    processPath=os.path.expanduser('~/HHbbgg_ETH_devel/outfiles/')+outTag+'/'+utils.IO.backgroundName[iProcess].split("/")[len(utils.IO.backgroundName[7].split("/"))-1].replace("output_","").replace(".root","")+"_preselection_diffNaming"+".root"
    if "GluGluToHHTo2B2G_node_"in processPath:
        treeName = "reducedTree_sig_node_"+str(iProcess-6)
    else:
        treeName = "reducedTree_bkg_"+str(iProcess)
    
    postprocessing.saveTree(processPath,dictVar,nCleaned,Y_pred_bkg[iSample],nameTree=treeName)
    
    processPath=os.path.expanduser('~/HHbbgg_ETH_devel/outfiles/')+outTag+'/'+utils.IO.backgroundName[iProcess].split("/")[len(utils.IO.backgroundName[7].split("/"))-1].replace("output_","").replace(".root","")+"WS_highMass.root"
    #it's a stupid way to save also mva output after cuts, more efficient solutions should be found
    Y_pred_total = np.column_stack((nCleaned,Y_pred_bkg[iSample]))
    
    nCleaned_MVA_2 = Y_pred_total[np.where(Y_pred_bkg[iSample]>MVA_Cut)]
    #

    nCleaned_massWindow_2 = postprocessing.cutInvariantMass(nCleaned_MVA_2,dictVar['diphotonCandidate.M()'],100,180)

    nCleaned_mdijet_2 = postprocessing.cutInvariantMass(nCleaned_massWindow_2,dictVar['dijetCandidate.M()'],100.,140.)
    nCleaned_MX = postprocessing.applyCut(nCleaned_mdijet_2,dictVar['MX'],350)


    Y_pred_cut=nCleaned_MX[:,nCleaned_MX.shape[1]-1].astype(np.float64)


    postprocessing.saveTree(processPath,dictVar,nCleaned_MX,Y_pred_cut,nameTree=treeName)

Processing sample: 0
nCleaned
(1900684, 18)
(23612, 18)
(1121, 18)
DiPhotonJetsBox_MGG-80toInf_13TeV-Sherpa events with 36 fb-1: 22.0267648609
(345, 18)
after cutting in dijet mass:6.66509161196
Processing sample: 3
nCleaned
(56382, 18)
(1255, 18)
(1080, 18)
GluGluHToGG_M-125_13TeV_powheg_pythia8 events with 36 fb-1: 3.77306576163
(337, 18)
after cutting in dijet mass:1.13964943105
Processing sample: 4
nCleaned
(76431, 18)
(1962, 18)
(1680, 18)
VBFHToGG_M-125_13TeV_powheg_pythia8 events with 36 fb-1: 0.511538008306
(488, 18)
after cutting in dijet mass:0.149276500263
Processing sample: 5
nCleaned
(57455, 18)
(3539, 18)
(2867, 18)
VHToGG_M125_13TeV_amcatnloFXFX_madspin_pythia8 events with 36 fb-1: 1.59166829387
(758, 18)
after cutting in dijet mass:0.38684968018
Processing sample: 6
nCleaned
(106395, 18)
(9732, 18)
(7922, 18)
bbHToGG_M-125_4FS_yb2_13TeV_amcatnlo events with 36 fb-1: 0.453958529657
(2806, 18)
after cutting in dijet mass:0.159626064415
Processing sample: 7
nCleaned
(26886

In [116]:
data_count_df = rpd.read_root(utils.IO.dataName[0],"bbggSelectionTree", columns = branch_names+additionalCut_names)

nTot,dictVar = postprocessing.stackFeatures(data_count_df,branch_names+additionalCut_names,isData=1)


#apply isSignal cleaning
nCleaned = nTot[np.where(nTot[:,dictVar['weight']]!=0),:][0]
print "nCleaned"
print nCleaned.shape
print Y_pred_data.shape

#save preselection data
processPath=os.path.expanduser('~/HHbbgg_ETH_devel/outfiles/')+outTag+'/'+utils.IO.dataName[0].split("/")[len(utils.IO.dataName[0].split("/"))-1].replace("output_","").replace(".root","")+"_preselection"+".root"
postprocessing.saveTree(processPath,dictVar,nCleaned,Y_pred_data)

processPath=os.path.expanduser('~/HHbbgg_ETH_devel/outfiles/')+outTag+'/'+utils.IO.dataName[0].split("/")[len(utils.IO.dataName[0].split("/"))-1].replace("output_","").replace(".root","")+"_preselection_diffNaming"+".root"
postprocessing.saveTree(processPath,dictVar,nCleaned,Y_pred_data,nameTree="reducedTree_bkg")

#do blinding
nCleaned_MVA = nCleaned[np.where(Y_pred_data>MVA_Cut)]

nCleaned_massWindow = nCleaned_MVA[np.where(np.logical_or(nCleaned_MVA[:,dictVar['diphotonCandidate.M()']]<115,nCleaned_MVA[:,dictVar['diphotonCandidate.M()']]>135))]
#    nCleaned_massWindow = nCleaned_massWindowDown[np.where(nCleaned_massWindowDown[:,0]>135)]
nCleaned_mdijet = nCleaned_massWindow[np.where(np.logical_or(nCleaned_massWindow[:,dictVar['dijetCandidate.M()']]<100,nCleaned_massWindow[:,dictVar['dijetCandidate.M()']]>140))]
#    nCleaned_mdijet = nCleaned_mdijet_down[np.where(nCleaned_mdijet_down[:,2]>140)]


processPath=os.path.expanduser('~/HHbbgg_ETH_devel/outfiles/')+outTag+'/'+utils.IO.dataName[0].split("/")[len(utils.IO.dataName[0].split("/"))-1].replace("output_","").replace(".root","")+"_blinded"+".root"
postprocessing.saveTree(processPath,dictVar,nCleaned_mdijet)

processPath=os.path.expanduser('~/HHbbgg_ETH_devel/outfiles/')+outTag+'/'+"BkgWS.root"
postprocessing.saveTree(processPath,dictVar,nCleaned,Y_pred_data,nameTree="reducedTree_bkg")

processPath=os.path.expanduser('~/HHbbgg_ETH_devel/outfiles/')+outTag+'/'+"BkgWS_highMass.root"
Y_pred_total = np.column_stack((nCleaned,Y_pred_data))

nCleaned_MVA_2 = Y_pred_total[np.where(Y_pred_data>MVA_Cut)]
print "MVA"
print nCleaned_MVA_2.shape
#    nCleaned_massWindow_2 = nCleaned_MVA_2[np.where(np.logical_or(nCleaned_MVA_2[:,dictVar['diphotonCandidate.M()']]<115,nCleaned_MVA_2[:,dictVar['diphotonCandidate.M()']]>135))]
#    nCleaned_mdijet_2 = postprocessing.cutInvariantMass(nCleaned_massWindow_2,dictVar['dijetCandidate.M()'],100.,140.)

nCleaned_mdijet_2 = postprocessing.cutInvariantMass(nCleaned_MVA_2,dictVar['dijetCandidate.M()'],100.,140.)
nCleaned_MX = postprocessing.applyCut(nCleaned_mdijet_2,dictVar['MX'],350)


Y_pred_cut=nCleaned_MX[:,nCleaned_MX.shape[1]-1].astype(np.float64)


postprocessing.saveTree(processPath,dictVar,nCleaned_MX,Y_pred_cut,nameTree="reducedTree_bkg")

nCleaned
(92390, 18)
(92390,)
MVA
(936, 19)
